In [ ]:
import pandas as pd
import numpy as np
from scipy.special import softmax
import copy
import json
import pickle
from tqdm import tqdm_notebook

In [ ]:
### define DDMM model class

class DDMM():
    
    #Miscellaneous function to speed up computation (i.e. by avoiding for loops)
    def multirange(self, counts):
        counts = np.asarray(counts)
        # Remove the following line if counts is always strictly positive.
        counts = counts[counts != 0]
        counts1 = counts[:-1]
        reset_index = np.cumsum(counts1)

        incr = np.ones(counts.sum(), dtype=int)
        incr[0] = 0
        incr[reset_index] = 1 - counts1

        # Reuse the incr array for the final result.
        incr.cumsum(out=incr)
        return np.tile(incr, (self.K,1)).T        
    
    # DDMM implementation
    def __init__(self, K, T, vocab, bows, alpha = None, beta = None):
        
        self.K = K
        self.vocab = vocab
        self.T = T
        self.bows = bows
        if alpha == None:
            self.alpha = 1.3
        else:
            self.alpha = alpha
        if beta == None:
            self.beta = 0.02
        else:
            self.beta = beta
        
        # create cluster assignment for each time step
        self.clust_asgn = []
        
        # create number of epochs to convergence for each time step
        self.epoch_conv = []
        
        # create epoch similarity for each time step
        self.epoch_similarity = []
        
        # create alphas
        self.alphas = []
        
        # create betas
        self.betas = []
        
        # create number of large enough clusters
        self.n_large_clusts = []
        
    def fit(self, max_epochs = 5, conv_crit = 0.97, lbda = 1, mu = 1):
        
        #initialize vocab length and hyperparameters
        V = len(self.vocab)
        alpha_ = self.alpha * np.ones(self.K)
        beta_ = self.beta * np.ones((self.K, V))
        
        #iterate through each timestep
        for t in range(self.T):
            
            n_large_clust = [self.K]
            
            alpha_copy = copy.deepcopy(alpha_)
            beta_copy = copy.deepcopy(beta_)
            self.alphas.append(alpha_copy) #update alphas
            self.betas.append(beta_copy) #update beta
            
            bow = self.bows[t] #get bow for time t
            bow_np = np.array(bow) #convert to numpy array
            D = bow.shape[0] #get document size
            clusters = np.random.choice(self.K, D) #cluster initialization
            
            #compute mk, nk and nkv for each k,v
            mk = np.bincount(clusters, minlength = self.K)
            nk = np.bincount(clusters, weights = np.sum(bow, axis = 1), minlength = self.K).astype(int)
            nkv = np.apply_along_axis(lambda x: np.bincount(clusters, weights = x, 
                                                            minlength = self.K), axis = 0, arr = bow)
            
            #number of words per document
            Nd = np.sum(bow, axis = 1)
            
            #initialize number of epochs and within-epoch similarity (to compare with conv_crit)
            n_epoch = 0
            epoch_sim = 0
            
            #repeat until convergence or maximum epochs
            while n_epoch < max_epochs and epoch_sim < conv_crit:
                print('Starting epoch', n_epoch + 1, 'for time step', t)
                curr_clust = copy.deepcopy(clusters) #take note of current cluster
                
                for d in range(D):
                    
                    #knock out d from its cluster; let say z is d's current cluster
                    z = clusters[d] #get d's current cluster
                    #modify mk, nk and nkv
                    mk[z] -= 1
                    nk[z] -= Nd[d]
                    nkv[z] -= bow_np[d]
                    
                    #sample a new cluster for d from Equation 15 (use log form)
                    #ignore sum(alpha_k) + D - 1 since it does not depend on k

                    #first component: log of alpha_k + m_k
                    first = np.log(alpha_ + mk)

                    #second component: sum from v=1 to V; j = 1 to Ndv of log(beta_kv + n_kv + j - 1)
                    temp = (beta_ + nkv).T
                    rep = bow_np[d]
                    temp1 = np.vstack(temp)
                    temp1 = temp1[np.repeat(np.arange(temp1.shape[0]), rep)]
                    temp2 = self.multirange(rep)
                    second = np.sum(np.log(temp1 + temp2), axis = 0)

                    #third component: sum from i=1 to Nd of log(sum of beta_kv for each v + n_k + i - 1)
                    Ndd = Nd[d]
                    temp = np.sum(beta_, axis = 1) + nk
                    third = np.sum(np.log(np.tile(temp, (Ndd, 1)) + \
                                          np.tile(np.arange(0, Ndd, 1), (self.K, 1)).T), axis = 0)

                    #sample a cluster from the proportions; let the cluster be q
                    prop = softmax(first + second - third)
                    q = np.random.choice(np.arange(self.K), p = prop) #new cluster assignment for d

                    #update mq, nq and nqv
                    mk[q] += 1
                    nk[q] += Nd[d]
                    nkv[q] += bow_np[d]

                    #update cluster assignment
                    clusters[d] = q
                
                n_epoch += 1 #update number of epoch
                epoch_sim = np.sum(curr_clust == clusters)/D #update cluster similarity for this epoch
                
                clust_prop = np.bincount(clusters, minlength = self.K)/D
                n_large_clust.append(len(clust_prop[clust_prop > 0.01]))
                
            #update tracker variables
            self.clust_asgn.append(clusters)
            self.epoch_conv.append(n_epoch)
            self.epoch_similarity.append(epoch_sim)
            
            #update alpha and beta
            
            #compute mk, nk and nkv for each k,v
            mk = np.bincount(clusters, minlength = self.K)
            nk = np.bincount(clusters, weights = np.sum(bow, axis = 1), minlength = self.K).astype(int)
            nkv = np.apply_along_axis(lambda x: np.bincount(clusters, weights = x, 
                                                            minlength = self.K), axis = 0, arr = bow)
            
            #number of words per document
            Nd = np.sum(bow, axis = 1)
            
            #modify alpha and beta
            alpha_ += lbda * mk
            beta_ += mu * nkv
            
            #update number of large clusters
            self.n_large_clusts.append(n_large_clust)

    def cluster_size(self):
        return [np.bincount(x, minlength = self.K) for x in self.clust_asgn]
    
    def top_words(self, n = 10):
        result = {}
        for t in range(self.T):
            bow = self.bows[t]
            clusters = self.clust_asgn[t]
            result_t = []
            for k in range(self.K):
                result_t.append(np.array(self.vocab)[np.array(bow.iloc[np.where(clusters == k)].sum(axis = 0) \
                                                    .sort_values()[::-1].index[:n]).astype(int)])
            result[t] = result_t
        return result

In [ ]:
version_name = 'title_tags_description'
print('Training for {}'.format(version_name))
meta = pd.read_csv(version_name + '/' + version_name + '_meta.csv', lineterminator='^')

# Read index to word json file
### Warning: the keys (word indices) are string type
with open(version_name + '/' + version_name + '_indexword.json') as json_file:
    vocab = json.load(json_file)

vocab = list(vocab.values())

# Read BoW file
bows = np.load(version_name + '/' + version_name + '_bow.npy')

T = len(np.unique(meta['t_month']))

bows = [pd.DataFrame(bows[meta[meta['t_month']==i].index]) for i in range(T)]

ddmm = DDMM(K=16, T=T, vocab=vocab, bows=bows)
ddmm.fit()
with open(version_name + '/' + version_name + '_oddmm_' + str(16) + '.pkl', 'wb') as output:
    pickle.dump(ddmm, output, pickle.HIGHEST_PROTOCOL)